# Day 21: Dirac Dice

There's not much to do as you slowly descend to the bottom of the ocean. The submarine computer challenges you to a nice game of Dirac Dice.

This game consists of a single die, two pawns, and a game board with a circular track containing ten spaces marked 1 through 10 clockwise. Each player's starting space is chosen randomly (your puzzle input). Player 1 goes first.

Players take turns moving. On each player's turn, the player rolls the die three times and adds up the results. Then, the player moves their pawn that many times forward around the track (that is, moving clockwise on spaces in order of increasing value, wrapping back around to 1 after 10). So, if a player is on space 7 and they roll 2, 2, and 1, they would move forward 5 times, to spaces 8, 9, 10, 1, and finally stopping on 2.

After each player moves, they increase their score by the value of the space their pawn stopped on. Players' scores start at 0. So, if the first player starts on space 7 and rolls a total of 5, they would stop on space 2 and add 2 to their score (for a total score of 2). The game immediately ends as a win for any player whose score reaches at least 1000.

Since the first game is a practice game, the submarine opens a compartment labeled deterministic dice and a 100-sided die falls out. This die always rolls 1 first, then 2, then 3, and so on up to 100, after which it starts over at 1 again. Play using this die.

## Game

In [ ]:
static var dice = 0;

static int Roll() => dice++ % 100 + 1;

display((Roll(), Roll(), Enumerable.Range(0, 100).Select(l => Roll()).ToArray().Skip(97)));

record struct Player(int pos) {
    public int score = 0;

    public bool Move() {
        var roll = Roll() + Roll() + Roll();
        pos = (pos - 1 + roll) % 10 + 1;
        score = score + pos;

        return score >= 1000;
    }
}

display(dice);

var p = new Player(7);
(p, 7 + 3 + 4 + 5, p.Move(), p)

Item1,Item2,Item3
1,2,"[ 100, 1, 2 ]"


102

Item1,Item2,Item3,Item4
"{ Player { pos = 7, score = 0 }: pos: 7, score: 0 }",19,False,"{ Player { pos = 9, score = 9 }: pos: 9, score: 9 }"


## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-21.sample");
var input = File.ReadAllLines(@"day-21.input");

var starting = input.Select(l => int.Parse(l.Split(':')[1])).ToArray();

starting

index,value
0,4
1,5


## Part 1

Play a practice game using the deterministic 100-sided die. The moment either player wins, what do you get if you multiply the score of the losing player by the number of times the die was rolled during the game?

In [ ]:
var game = starting.Select(l => new Player(l)).ToArray();
display(game);

dice = 0;

while (!game[0].Move() && !game[1].Move()) ;

dice * game.First(p => p.score < 1000).score

index,pos,score
0,4,0
1,5,0


864900

## Part 2

Now that you're warmed up, it's time to play the real game.

A second compartment opens, this time labeled Dirac dice. Out of it falls a single three-sided die.

As you experiment with the die, you feel a little strange. An informational brochure in the compartment explains that this is a quantum die: when you roll it, the universe splits into multiple copies, one copy for each possible outcome of the die. In this case, rolling the die always splits the universe into three copies: one where the outcome of the roll was 1, one where it was 2, and one where it was 3.

The game is played the same as before, although to prevent things from getting too far out of hand, the game now ends when either player's score reaches at least 21.

Using your given starting positions, determine every possible outcome. Find the player that wins in more universes; in how many universes does that player win?

In [ ]:
static (int roll, int num)[] threeRolls = new [] { 
        (3,1), (4,3), (5,6), (6,7), (7,6), (8,3), (9,1),
};
const int splits = 27;
const int winAfter = 20;

record Dirac(int pos, int score = 0, int splits = 1, long count = 1) {
    public Dirac[] Move()
        => threeRolls
            .Select(r => {
                var next = (pos - 1 + r.roll) % 10 + 1;
                return new Dirac(next, score + next, r.num, count * r.num);
            }).ToArray();
    public Dirac Split(int times)
        => new Dirac(pos, score, times, count * times);
}

class Game {
    public Dirac[] A { get; private set; }
    public Dirac[] B { get; private set; }

    public Game(int[] starting) {
        A = new[] { new Dirac(starting[0]) };
        B = new[] { new Dirac(starting[1]) };
    }

    public long winsA;
    public long winsB;

    long Wins(Dirac[] moves, Dirac[] other)
        => moves.Where(p => p.score > winAfter).Sum(p => p.count)
            * other.Sum(p => p.count);

    public bool Play() {
        var movesA = A.SelectMany(p => p.Move()).ToArray();
        winsA = winsA + Wins(movesA, B);
        A = movesA.Where(p => p.score <= winAfter).ToArray();

        var movesB = B.SelectMany(p => p.Move()).ToArray();
        winsB = winsB + Wins(movesB, A);
        B = movesB.Where(p => p.score <= winAfter).ToArray();

        return A.Any() && B.Any();
    }

    public long[] State
        => new[] {A.Sum(p => p.count), B.Sum(p => p.count), winsA, winsB};
}

var game = new Game(starting);

while(game.Play());

game.State

index,value
0,0
1,0
2,575111835924670
3,392525387812463
